In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [3]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [4]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [5]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [6]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [7]:
def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://www.worldhistory.org/article/926/magic-in-ancient-greece/")

**Summary of Magic in Ancient Greece**

Magic in ancient Greece refers to the practice of spells, potions, and amulets intended to influence the world of humans. The Greeks believed that magic could be used to cure physical ailments, improve one's sex life, or protect against evil spirits. These magical practices were often associated with Greek mythology, where figures like Hecate, Hermes, and Circe played important roles.

### History of Magic in Greece

*   **Mythological References**: The use of magic appears in ancient Greek mythological tales, such as those involving Hecate, Hermes, and Circe.
*   **Curses**: Curse tablets were used to maintain public order by threatening magical punishment for behaviors deemed detrimental to the community.
*   **Magic Spells**: Surviving papyri from ancient Greece show a range of spells and incantations, including recipes for amulets and poisons.

### Types of Magic

*   **Amulets**: Small objects believed to possess magical powers, such as talismans that brought good luck or phylacteries that protected.
*     Curse Tablets
*   Curse tablets were inscribed sheets of metal rolled or folded, sometimes nailed shut and buried in the ground.

### Influencers of Magic

*   **Hecate**: A goddess associated with witchcraft, magic, and the supernatural.
*   **Orpheus and Hermes**: Figures believed to possess magical powers and influence over the lives of individuals.